In [ ]:
trainpath = 'base_dir/train_dir'
validpath = 'base_dir/val_dir'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
datagen = ImageDataGenerator(preprocessing_function= tensorflow.keras.applications.mobilenet.preprocess_input)
train_batches = datagen.flow_from_directory(train_path,target_size=(image_size,image_size),batch_size=train_batch_size)
valid_batches = datagen.flow_from_directory(valid_path,target_size=(image_size,image_size),batch_size=val_batch_size)
test_batches = datagen.flow_from_directory(valid_path,target_size=(image_size,image_size),batch_size=1,shuffle=False)

In [ ]:
mob = tensorflow.keras.applications.mobilenet.MobileNet()
x = mob.layers[-6].output
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
for layer in model.layers[:-23]:
    layer.trainable = False

Melanoma class is provided with higher weights as this is the most deadliest skin cancer type.

In [ ]:
class_weights={0: 1.0,1: 1.0,2: 1.0,3: 1.0,4: 3.0,5: 1.0,6: 1.0}

In [ ]:
filepath = "model.h5"

In [ ]:
CP = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1,save_best_only=True, mode='max')

reducelr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [CP, reducelr]

history = model.fit_generator(train_batches, steps_per_epoch=train_steps,class_weight=class_weights,validation_data=valid_batches,validation_steps=val_steps,epochs=30, verbose=1,callbacks=callbacks_list)

In [ ]:
model.load_weights('model.h5')
model.evaluate_generator(test_batches,steps=len(df_val))